In [7]:
import neo4j
import pandas as pd
from utils_neo4j import init, add_station_relationships
import pandas as pd
import ast

In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [3]:
# load departures preproc from json
departures_preproc = pd.read_csv("data_geofox/departures_all_preproc2.csv")

In [8]:
# Convert departure_times from string to list
if 'departure_times' in departures_preproc.columns:
    departures_preproc['departure_times'] = departures_preproc['departure_times'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

# Convert arrival_times from string to list
if 'arrival_times' in departures_preproc.columns:
    departures_preproc['arrival_times'] = departures_preproc['arrival_times'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

In [9]:
departures_preproc

,from_station,from_station_id,to_station,line_unique,line_name,line_info,departure_times,arrival_times,duration
0,A Burgwedel,Master:86007,Basselweg,HHA-B:603_HHA-B#Grothwisch#Rathausmarkt#1,603,Niederflur-Nachtbus,[2025-04-03 01:02:00],[2025-04-03 01:03:00],1
1,A Burgwedel,Master:86007,Freizeitbad Bondenwald,HHA-B:5_HHA-B#A Burgwedel#Hauptbahnhof/ZOB#1,5,Niederflur Metrobus,"[2025-04-03 07:05:00, 2025-04-03 07:15:00, 202...","[2025-04-03 07:06:00, 2025-04-03 07:16:00, 202...",1
2,A Burgwedel,Master:86007,Graf-Otto-Weg,HHA-B:603_HHA-B#Rathausmarkt#Grothwisch#6,603,Niederflur-Nachtbus,"[2025-04-03 01:56:00, 2025-04-03 02:26:00, 202...","[2025-04-03 01:57:00, 2025-04-03 02:27:00, 202...",1
3,A Eidelstedt Zentrum,Master:83013,Behrmannplatz,VHH:281_VHH#S Krupunder#U Lattenkamp#1,281,Niederflur Stadtbus,"[2025-04-03 07:26:00, 2025-04-03 07:46:00, 202...","[2025-04-03 07:27:00, 2025-04-03 07:48:00, 202...",1
4,A Eidelstedt Zentrum,Master:83013,Behrmannplatz,VHH:281_VHH#UK Eppendorf (Haupteingang)#S Krup...,281,Niederflur Stadtbus,"[2025-04-03 07:22:00, 2025-04-03 07:42:00, 202...","[2025-04-03 07:23:00, 2025-04-03 07:43:00, 202...",1
...,...,...,...,...,...,...,...,...,...
23803,Überseering (West),Master:70017,Goebenstraße,HHA-B:20_HHA-B#S Rübenkamp#Bf. Altona#6,20,Niederflur Metrobus,"[2025-04-03 09:15:00, 2025-04-03 09:25:00, 202...","[2025-04-03 09:16:00, 2025-04-03 09:26:00, 202...",1
23804,Überseering (West),Master:70017,Heinrich-Traun-Platz,HHA-B:179_HHA-B#U Borgweg#S Poppenbüttel (Went...,179,Niederflur Stadtbus,[2025-04-03 19:30:00],[2025-04-03 19:31:00],1
23805,Überseering (West),Master:70017,Jahnring (Mitte),HHA-B:20_HHA-B#S Rübenkamp#Winterhuder Marktpl...,20,Niederflur Metrobus,[2025-04-03 23:59:00],[2025-04-04 00:00:00],1
23806,Überseering (West),Master:70017,Kottwitzstraße,HHA-B:20_HHA-B#Bf. Altona#S Rübenkamp#1,20,Niederflur Metrobus,"[2025-04-03 06:54:00, 2025-04-03 07:04:00, 202...","[2025-04-03 06:55:00, 2025-04-03 07:05:00, 202...",1


In [10]:
processed_data = departures_preproc.to_dict('records')

In [11]:
# Execute the query
with driver.session() as session:
    add_station_relationships(processed_data, session)

driver.close()

C:\Users\paul-\AppData\Local\Temp\ipykernel_9844\2045833411.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Adding 23808 consolidated relationships to Neo4j...


Adding relationships to Neo4j:  12%|█▏        | 28/239 [00:05<00:35,  5.96it/s]

Skipping relationship from 'Bf. Harburg (Großmoordamm)' to 'Neuhof (Fähre)' - Destination station does not exist


Adding relationships to Neo4j:  34%|███▍      | 81/239 [00:14<00:25,  6.26it/s]

...

Skipping relationship from 'U Rauhes Haus' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist


Adding relationships to Neo4j:  96%|█████████▌| 229/239 [00:36<00:01,  6.29it/s]

Skipping relationship from 'Weidendamm' to 'Neuhof (Fähre)' - Destination station does not exist


Adding relationships to Neo4j: 100%|██████████| 239/239 [00:38<00:00,  6.27it/s]

  - 25 due to missing destination stations
Creating additional indexes for optimized querying...
Relationship creation complete!
